In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from tqdm import tqdm

In [2]:
games =  pd.read_parquet('../data/cleaned/games_cleaned.parquet')
test_df = pd.read_parquet('../data/cleaned/test_recommendations.parquet')
train_df = pd.read_parquet('../data/cleaned/train_recommendations.parquet')

In [4]:
unique_users = train_df['user_id'].unique()
unique_items = games['app_id'].unique()

user_to_idx = {uid: idx for idx, uid in enumerate(unique_users)}
item_to_idx = {iid: idx for idx, iid in enumerate(unique_items)}
idx_to_user = {idx: uid for uid, idx in user_to_idx.items()}
idx_to_item = {idx: iid for iid, idx in item_to_idx.items()}

In [5]:
games['content'] = games['title'] + ' ' + games['description'] + ' ' + games['tags'].apply(lambda tags: ' '.join(tags))

In [6]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(games['content'])

In [7]:
with open('../models/tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

# Funzione per raccomandare giochi simili con le distanze (similarità)
def recommend_content_based(game_id, N=10):
    
    game_idx = item_to_idx[game_id]
    
    # Estrai il vettore del gioco per cui vogliamo le raccomandazioni
    game_vector = tfidf_matrix[game_idx]
    
    # Calcola la similarità coseno tra il vettore del gioco richiesto e tutti gli altri
    similarity_scores = cosine_similarity(game_vector.reshape(1, -1), tfidf_matrix)[0]
    
    # Ordina i giochi per similarità decrescente
    similar_indices = similarity_scores.argsort()[::-1][1:N+1]  # Ignora il gioco stesso
    
    # Ottieni gli app_id dei giochi più simili e le loro distanze (similarità)
    recommended_game_ids = games['app_id'].iloc[similar_indices].tolist()
    recommended_distances = similarity_scores[similar_indices].tolist()
    
    return recommended_game_ids, recommended_distances

In [9]:
# Ora fuori dalla funzione, otteniamo i nomi dei giochi:
current_game_name = games[games['app_id'] == 1078730]['title'].values[0]
recommended_games, recommended_distances = recommend_content_based(1078730, N=10)
# Ottieni i nomi dei giochi raccomandati
recommended_game_names = games[games['app_id'].isin(recommended_games)]['title'].tolist()

# Stampa il risultato
print(f"Game being considered: {current_game_name}")
print("Recommended games:")
for game in recommended_game_names:
    print(f"- {game}")

Game being considered: Soccer Manager 2020
Recommended games:
- Pro 11 - Football Manager Game
- Soccer Manager 2021
- WE ARE FOOTBALL
- New Star Manager
- Soccer Manager 2022
- Football Manager 2023
- Futuball - Future Football Manager Game
- Club Manager 2016
- Football Manager 2020 In-game Editor
- Crazy Soccer: Football Stars


In [10]:
def get_recommendations_for_user(user_id, N=10):
    recommended_games_dict = {}

    # Itera su tutti i giochi con cui l'utente ha interagito
    for game_id in train_df[train_df['user_id'] == user_id]['app_id']:
        # Ottieni le raccomandazioni per il gioco corrente
        recommended_games, recommended_similarity = recommend_content_based(game_id, N=10)
        
        # Aggiungi le raccomandazioni al dizionario, escludendo il gioco stesso
        for rec_game_id, similarity in zip(recommended_games, recommended_similarity):
            if rec_game_id != game_id:  # Escludi il gioco stesso
                if rec_game_id not in recommended_games_dict:
                    recommended_games_dict[rec_game_id] = similarity
                else:
                    # Mantieni solo la similarità più alta per ogni gioco
                    recommended_games_dict[rec_game_id] = max(recommended_games_dict[rec_game_id], similarity)

    # Ordina i giochi in base alla similarità e prendi i primi N
    sorted_recommendations = sorted(recommended_games_dict.items(), key=lambda x: x[1], reverse=True)

    # Torna i primi N giochi distinti
    top_n_games = sorted_recommendations[:N]
    
    return top_n_games

In [11]:
# Testa la funzione
user_id = 8392451  # Usa un esempio di user_id
recommended_games = get_recommendations_for_user(user_id, N=10)
for game, similarity in recommended_games:
    print(f"Game ID: {game}, Similarity: {similarity:.4f}")

Game ID: 49520, Similarity: 1.0000
Game ID: 744680, Similarity: 0.8958
Game ID: 1390710, Similarity: 0.8750
Game ID: 1283090, Similarity: 0.8111
Game ID: 330830, Similarity: 0.7590
Game ID: 2199500, Similarity: 0.7360
Game ID: 801860, Similarity: 0.7320
Game ID: 1517481, Similarity: 0.7268
Game ID: 1946960, Similarity: 0.7126
Game ID: 1454970, Similarity: 0.7120


In [12]:
# Ottieni gli utenti presenti nel test set e nel train set come set per una ricerca più veloce
test_users_set = set(test_df['user_id'].unique())
train_users_set = set(train_df['user_id'].unique())

In [13]:
known_users = list(test_users_set & train_users_set)  # Intersezione come lista
cold_start_users = list(test_users_set - train_users_set)

In [14]:
# Conta il numero di righe per ogni user_id nel test set
user_counts = test_df.groupby('user_id').size()

# Filtra gli utenti che hanno almeno 10 righe
known_users_filtered = [user_id for user_id in known_users if user_counts[user_id] >= 10]

# Stampa gli utenti filtrati
print(f"Utenti noti con almeno 10 righe nel test set: {len(known_users_filtered)}")


Utenti noti con almeno 10 righe nel test set: 15029


In [15]:
import numpy as np

# Funzione per calcolare la precisione
def calculate_precision(user_id, recommended_items, test_df):
    # Ottieni gli articoli effettivi con cui l'utente ha interagito nel test set (dove is_recommended == 1)
    actual_items = test_df[(test_df['user_id'] == user_id) & (test_df['is_recommended'] == 1)]['app_id']
    
    # Confronta gli articoli raccomandati con quelli effettivi
    recommended_item_ids = [item[0] for item in recommended_items]
    
    # Calcola la precisione come frazione di articoli raccomandati che sono effettivamente rilevanti
    relevant_recommendations = sum(1 for item in recommended_item_ids if item in actual_items.values)
    precision = relevant_recommendations / len(recommended_item_ids) if len(recommended_item_ids) > 0 else 0
    
    return precision

# Funzione per calcolare il recall
def calculate_recall(user_id, recommended_items, test_df):
    # Ottieni gli articoli effettivi con cui l'utente ha interagito nel test set (dove is_recommended == 1)
    actual_items = test_df[(test_df['user_id'] == user_id) & (test_df['is_recommended'] == 1)]['app_id']
    
    # Confronta gli articoli raccomandati con quelli effettivi
    recommended_item_ids = [item[0] for item in recommended_items]
    
    # Calcola il recall come frazione di articoli rilevanti che sono stati raccomandati
    relevant_recommendations = sum(1 for item in recommended_item_ids if item in actual_items.values)
    recall = relevant_recommendations / len(actual_items) if len(actual_items) > 0 else 0
    
    return recall

# Funzione per calcolare il F1-Score
def calculate_f1_score(user_id, recommended_items, test_df):
    precision = calculate_precision(user_id, recommended_items, test_df)
    recall = calculate_recall(user_id, recommended_items, test_df)
    
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def calculate_hit_ratio(user_id, recommended_items, test_df, N=10):
    actual_items = test_df[(test_df['user_id'] == user_id) & (test_df['is_recommended'] == 1)]['app_id']
    recommended_item_ids = [item[0] for item in recommended_items][:N]  # Limita alle prime N raccomandazioni
    hit = 1 if any(item in actual_items.values for item in recommended_item_ids) else 0
    return hit

user_id = 22  # esempio di user_id nel test set
recommended_items = get_recommendations_for_user(user_id)

# Calcola la precisione, recall e F1-Score
precision = calculate_precision(user_id, recommended_items, test_df)
recall = calculate_recall(user_id, recommended_items, test_df)
f1 = calculate_f1_score(user_id, recommended_items, test_df)
hit = calculate_hit_ratio(user_id, recommended_items, test_df)

# Stampa i risultati
print(f"Precisione per l'utente {user_id}: {precision}")
print(f"Recall per l'utente {user_id}: {recall}")
print(f"F1-Score per l'utente {user_id}: {f1}")
print(f"Hit Ratio per l'utente {user_id}: {hit}")


Precisione per l'utente 22: 0.0
Recall per l'utente 22: 0.0
F1-Score per l'utente 22: 0
Hit Ratio per l'utente 22: 0


In [16]:
known_user_metrics = []
for user_id in tqdm(known_users_filtered[0:100]):
    recommended_items = get_recommendations_for_user(user_id)
    precision = calculate_precision(user_id, recommended_items, test_df)
    recall = calculate_recall(user_id, recommended_items, test_df)
    f1 = calculate_f1_score(user_id, recommended_items, test_df)
    hit = calculate_hit_ratio(user_id, recommended_items, test_df)
    known_user_metrics.append((user_id, precision, recall, f1, hit))

100%|██████████| 100/100 [01:12<00:00,  1.38it/s]


In [17]:
# Calcola la media delle metriche per gli utenti noti
if len(known_user_metrics) > 0:
    avg_precision_known = np.mean([metric[1] for metric in known_user_metrics])
    avg_recall_known = np.mean([metric[2] for metric in known_user_metrics])
    avg_f1_known = np.mean([metric[3] for metric in known_user_metrics])
    avg_hit_known = np.mean([metric[4] for metric in known_user_metrics])
else:
    avg_precision_known, avg_recall_known, avg_f1_known = 0, 0, 0

# Stampa i risultati per gli utenti noti
print("Metriche per gli utenti noti:")
print(f"Precisione media: {avg_precision_known:.4f}")
print(f"Recall medio: {avg_recall_known:.4f}")
print(f"F1-Score medio: {avg_f1_known:.4f}")
print(f"Hit Ratio medio: {avg_hit_known:.4f}")

Metriche per gli utenti noti:
Precisione media: 0.0120
Recall medio: 0.0079
F1-Score medio: 0.0090
Hit Ratio medio: 0.1200
